In [2]:
## This block uses the buy and sell strategy extracted from the LSTM model
## it returns how the model would have performed in comparison to the traditional buy-and-hold strategy

# import libraries and open data
import pandas as pd
import numpy as np
df= pd.read_csv('sp500.csv')
pred = pd.read_csv('result_3.csv',header=None)

df=df[df['Date']>="1973-07-10"]  # set start date - 1 
df=df[df['Date']<="1974-12-31"]  # set end date 
df.drop(['Date','Open', 'High','Low','Close','Volume',], axis=1,inplace=True) # drop columns that aren't needed 


# convert to arrays
pred=pred.to_numpy()
df=df.to_numpy()
pred=pred.flatten()
df=df.flatten()

# if the output (probability of stock going up tomorrow) is greater than or equal to .5, it's a buy signal (1)
# if the output (probability of stock going up tomorrow) is less than .5, it's a sell signal (0)
for i in range(len(pred)):
    if pred[i]>=0.5:
        pred[i]=1
    else:
        pred[i]=0

# set total as the starting money
total=df[0]
# position is whether you are holding the stock or not. We start with not owning the stock but having just enough money to buy it
position=False


for i in range(len(pred)):
    # if predicted the market to go up and don't have the stock, buy it
    if pred[i]==1 and position==False:
        total=total-df[i]
        position=True
    # if predicted the market to go down and own the stock, sell it
    elif pred[i]==0 and position==True:
        total=total+df[i]
        position=False
# sell the stock on the last day if holding it
if position==True:
    total=total+df[-1]

print("Starting $:")
print(df[0]) 
print("Buy and hold at the end:")
print(df[-1])
print("lSTM at the end:")
print(total)

Starting $:
103.519997
Buy and hold at the end:
68.559998
lSTM at the end:
128.19001799999998


In [4]:
## this block returns the model performance in following terms:
## directional accuracy, + accuracy, - accuracy, and % of + prediction

# set counters
positive=0
negative=0
f_positive=0
f_negative=0

for i in range(len(pred)):
    # if predicted up and correct, positive 
    if pred[i]==1 and df[i+1]>=df[i]:
        positive=positive+1
    # if predcited up and wrong, f_positive
    elif pred[i]==1 and df[i+1]<df[i]:
        f_positive= f_positive+1
    # if predcited down and correct, negative
    elif pred[i]==0 and df[i+1]<df[i]:
        negative=negative+1
    # if predcited down and wrong, f_negative
    elif pred[i]==0 and df[i+1]>=df[i]:
        f_negative=f_negative+1

# return values
print ('directional accuracy: '+str((positive+negative)/len(pred)))
print ('+ accuracy: '+str(positive/(positive+f_positive)))
print ('- accuracy: '+str(negative/(negative+f_negative)))
print ('% of + prediction: '+str((positive+f_positive)/len(pred)))

directional accuracy: 0.6096256684491979
+ accuracy: 0.5725806451612904
- accuracy: 0.628
% of + prediction: 0.3315508021390374
